In [8]:
! wget https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/nWHh53OXDGoYqw
! unrar x nWHh53OXDGoYqw ./
! sudo rm nWHh53OXDGoYqw

--2020-03-26 08:22:58--  https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/nWHh53OXDGoYqw
Resolving getfile.dokpub.com (getfile.dokpub.com)... 78.46.92.107
Connecting to getfile.dokpub.com (getfile.dokpub.com)|78.46.92.107|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://downloader.disk.yandex.ru/disk/20aa228e1e2afd409d111ef0c634a59ca0d233fda3b471929a5031aff88f9c2b/5e7c9dab/W1bfCUTmXCWhdha9dKeAEG5xDU-aNMnIvjh7YdAotldF4XdPUrXlCbK91QoMyflaeRO5tpBNric5XXCsJdtCgg%3D%3D?uid=0&filename=workout-movement-counting.rar&disposition=attachment&hash=wOCQY8KOzBsDRRoxcwDmhn9UtyRehuo3P6fJL7dr8IzpjC6cgdyw8mk7tz7vBMSpq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=application%2Fx-rar&owner_uid=210071618&fsize=51397642&hid=8a1bf3ae2c0624b0fd012d68494bec26&media_type=compressed&tknv=v2 [following]
--2020-03-26 08:22:59--  https://downloader.disk.yandex.ru/disk/20aa228e1e2afd409d111ef0c634a59ca0d233fda3b471929a5031aff88f9c2b/5e7c9dab/W1bfCUTmXCWh

In [2]:
! pip install pyoptflow
! pip install opencv-python --force-reinstall

  Created wheel for pyoptflow: filename=pyoptflow-1.3.1-cp36-none-any.whl size=7335 sha256=d9d9cd1c5a0e90d8d3a571fa1764e4f5ea81f1596581027ece8ef4bf5364bd8d
  Stored in directory: /root/.cache/pip/wheels/e5/f2/3a/23d29693cf27ebc404cf0ffffc31b5b1226b4b8a369021be7e
Successfully built pyoptflow
     |████████████████████████████████| 28.2MB 110kB/s 
     |████████████████████████████████| 20.2MB 155kB/s 
ERROR: tensorflow-model-optimization 0.2.1 requires enum34~=1.1, which is not installed.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found exis

In [0]:
import cv2
import numpy as np
import time
import argparse

import matplotlib.pyplot as plt
import shutil
import os
from pyoptflow import LucasKanade

from torchvision import models
import torchvision.transforms as transforms
from torch import nn

from tqdm import tqdm_notebook

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.manifold import TSNE
import pandas as pd

import subprocess

%matplotlib inline

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
file_list = drive.ListFile({'q': "'root' in parents"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: flows, id: 1GMYGRY-_Yko0rHwbb5IcsFmWsT-sGcf9
title: OptFlowIMG_8266.zip, id: 1_A3QoeSCyOufiggMwim2mnE66FddfRqY
title: Копия Coronavirus – When Should You Close Your Office?, id: 10j1RmoFWu49jNHZOxTN9f_-nVSak7QZi6TMKXyScJ7k
title: Colab Notebooks, id: 1Cv1snHApMH6C596RWR4hImx5WhOeWvRg
title: Копия Таблица для планирования бюджета, id: 1Hy2U7EkETpEVGblUUunh6PP-Z0H31Weoclb4Mv-4u9g
title: Ставки_ДобрыйЧеловек, id: 1PKkK_bwQs_r2gVj7sv63GZg2IGTkQ3WBx4tjoSe_NCw
title: Реклместа, id: 1MA9lzAyHH8CEyRToBn5R6OAgNLh2kxgGCPc46EyCIis
title: graph description, id: 1cWSAyJ8EwyZbYH1PTVod3RVYrr-uRYSY9mxR6sJwEK8
title: wordtable, id: 1GG_0BpTqMA6NsteCYS-8pc_OAmNzvo0ZqUwz9hJ9TyE
title: график мест, id: 15Tc-5cVFEWVeBrQk4zKrVa03n5APgOBLSxMfH6hx_58


In [0]:
def captureOpticalFlow(fileType, file, takeFrame = 5, folder_id = '1GMYGRY-_Yko0rHwbb5IcsFmWsT-sGcf9'):

    folders = {'flow': f'{fileType}/',
              'frames': f'Frames{fileType}/'}

    shutil.rmtree(folders['flow'], ignore_errors=True)
    shutil.rmtree(folders['frames'], ignore_errors=True)
    os.mkdir(folders['flow'])
    os.mkdir(folders['frames'])

    cap = cv2.VideoCapture(file)
    IMG_SIZE = (128, 128)

    ret, first_frame = cap.read()
    first_frame = cv2.resize(first_frame, IMG_SIZE)

    prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(first_frame)
    mask[..., 1] = 255

    i = 0
    flow = []
    while cap.isOpened():
    try:
        ret, frame = cap.read()
        frame = cv2.resize(frame, IMG_SIZE)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculates dense optical flow by Farneback method
        # https://docs.opencv.org/3.0-beta/modules/video/doc/motion_analysis_and_object_tracking.html#calcopticalflowfarneback
        if len(flow) == 0:
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 100, 3, 7, 1.1, 0)
        else:
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, flow, 0.5, 3, 100, 3, 7, 1.1, 0)

        # Computes the magnitude and angle of the 2D vectors
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        # Sets image hue according to the optical flow direction
        mask[..., 0] = angle * 180 / np.pi / 2

        # Sets image value according to the optical flow magnitude (normalized)
        mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

        # Converts HSV to RGB (BGR) color representation
        rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)

        #cv.imshow("dense optical flow", rgb)
        if i % takeFrame == 0:
            cv2.imwrite(f'''{folders['flow']}/{i}.png''', rgb)
            #cv2.imwrite(f'''{folders['frames']}/{i}.png''', frame)
        i+=1
        prev_gray = gray
        # Frames are read by intervals of 1 millisecond. The programs breaks out of the while loop when the user presses the 'q' key
        except Exception as e:
            print(f'Programm run into {e}')
            process = subprocess.run(['zip', '-r', folders['flow'].split('/')[0] + '.zip', folders['flow']]) 

            file = drive.CreateFile({'parents':[{u'id': folder_id}]}) 
            file.SetContentFile(folders['flow'].split('/')[0] + '.zip')
            file.Upload()

            process = subprocess.run(['rm', '-r', folders['flow'] + '.zip', folders['flow'], folders['frames']]) 
            break

In [9]:
# capturing optical flow data
for video in tqdm_notebook(os.listdir('data/')):
    captureOpticalFlow(video.split('.')[0], 'data/' + video)
! sudo rm -r Frames*

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


KeyboardInterrupt: ignored

In [10]:
video = 'IMG_8266.MOV'
captureOpticalFlow(video.split('.')[0], 'data/' + video)
! sudo rm -r Frames*

Programm run into OpenCV(4.2.0) /io/opencv/modules/imgproc/src/resize.cpp:4045: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

